In [1]:
##Liberias

# Generales
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import glob

# limpieza y stopwords
import re
from nltk.corpus import stopwords

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

## 1. Datos

In [2]:
#extrayendo data que proviene de 1. Cleaning-Resoluciones-Completas

DF = pd.read_csv("Total-data-resol.csv")
DF

,Año,Categoría,Materia,Número de resolución,Empresa denunciada,Criterios aplicables,Medios probatoriosCargo,Medios probatorios Descargo,Cuestiones,Normativa,Multas,Medidas correctivas,Configuración del Tribunal,Decisión
0,2006,Pérdida de equipaje,Idoneidad del servicio,0064-2006/TDC-INDECOPI,Empresa de Transporte Flores Hermanos S.R.L.,la responsabilidad administrativa de Transport...,Boleta por servicio de encomienda,Denuncia policial por hurto,Competencia Fuerza mayor,Reglamento Nacional de Ferrocarriles D. Suprem...,"0,50 UIT Pago de costas y costos",Pago de S/.850,Juan Francisco Rojas Leo Julio Balazar Durand ...,1
1,2006,Pérdida de equipaje,Idoneidad del servicio,0071-2006/TDC-INDECOPI,PAREDES ESTRELLA CARGO S.A.,El punto de partida para demostrar la existenc...,0,0,"Competencia Fuerza mayor, caso fortuito",Ley No. 27181 Ley General de Transporte y Trán...,"0,5 UIT","Pago de S/.1,650",Juan Francisco Rojas Leo Julio Balazar Durand ...,1
2,2006,Pérdida de equipaje,Idoneidad del servicio Gradación de la sanción,0142-2006/TDC-INDECOPI,EMPRESA DE TRANSPORTES SANTA ÚRSULA S.A.C,"Para el dictado de medidas correctivas, la aus...",0,0,0,Reglamento Nacional de Ferrocarriles D. Suprem...,"0,50 UIT",Pago de S/.850,Juan Francisco Rojas Leo Julio Balazar Durand ...,0
3,2006,Pérdida de equipaje,Idoneidad del servicio,1335-2006/TDC-INDECOPI,Empresa de Transporte El Pino SAC (El Pino),la causa no imputable debe ser entendida como ...,Boleta de Venta,0,deber de custodia del equipaje,Ley de Aeronaútica Civil del Perú DECRETO LEGI...,"0,5 UIT Pago de costas y costos",Pago de S/. 966,Juan Francisco Rojas Leo Julio Balazar Durand ...,1
4,2006,Pérdida de equipaje,Idoneidad del servicio,1428-2006/TDC.INDECOPI,EMPRESA DE TRANSPORTES CARGO CAVASSA S.A.C.,Al contratar un servicio de transporte de carg...,0,Guia Boleta de Venta No. 51285,0,D. Supremo No. 009-2004-MTC Reglamento Naciona...,0,0,Juan Francisco Rojas Leo Julio Balazar Durand ...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,2019,Pérdida de equipaje,Idoneidad del servicio,1799-2019/SPC-INDECOPI,Empresa de Transporte Flores Hermanos S.R.L.,"""En virtud de lo expuesto, en el presente caso...",0,0,0,Código de Protección y Defensa del Consumidor ...,0,0,Javier Eduardo Raymundo Villa García Vargas Ju...,1
279,2019,Pérdida de equipaje,Idoneidad del servicio,3010-2019/SPC-INDECOPI,TRANSPORTES TURISMO OROPESA E.I.R.L.,"""Sobre el particular, esta Sala no coincide co...",0,0,0,Código de Protección y Defensa del Consumidor ...,0,0,Javier Eduardo Raymundo Villa García Vargas Ro...,0
280,2020,Pérdida de equipaje,Idoneidad del servicio,0002-2020/SPC-INDECOPI,EMPRESA DE TRANSPORTES TURISMO SEÑOR DE ATACO ...,"""De esta manera, al no haber quedado acreditad...",0,0,0,Ley No. 27444 Ley del Procedimiento Administra...,0,0,Javier Eduardo Raymundo Villa García Vargas Ju...,1
281,2020,Pérdida de equipaje,Idoneidad del servicio,0944-2020/SPC-INDECOPI,EMPRESA DE TRANSPORTES Y SERVICIOS MÚLTIPLES S...,"""Dicho esto, este Colegiado considera que el f...",0,0,0,Código de Protección y Defensa del Consumidor ...,50 UIT,0,Javier Eduardo Raymundo Villa García Vargas Ro...,1


In [3]:
## Se usaran 2 inputs para TF-IDF & Embedings

In [4]:
# Criterios aplicables y Decision

DF_criterios = DF[["Criterios aplicables", "Decisión"]]
DF_criterios

,Criterios aplicables,Decisión
0,la responsabilidad administrativa de Transport...,1
1,El punto de partida para demostrar la existenc...,1
2,"Para el dictado de medidas correctivas, la aus...",0
3,la causa no imputable debe ser entendida como ...,1
4,Al contratar un servicio de transporte de carg...,0
...,...,...
278,"""En virtud de lo expuesto, en el presente caso...",1
279,"""Sobre el particular, esta Sala no coincide co...",0
280,"""De esta manera, al no haber quedado acreditad...",1
281,"""Dicho esto, este Colegiado considera que el f...",1


In [5]:
#Todas las variables concatenadas

DF["todo"] = DF['Criterios aplicables'] + " " + DF["Empresa denunciada"] + " " + DF["Medios probatoriosCargo"] + " " + DF["Medios probatorios Descargo"] + " " + DF["Cuestiones"] + " " + DF["Normativa"] + " " + DF["Configuración del Tribunal"]

DF_todas = DF[["todo", "Decisión"]]
DF_todas

,todo,Decisión
0,la responsabilidad administrativa de Transport...,1
1,El punto de partida para demostrar la existenc...,1
2,"Para el dictado de medidas correctivas, la aus...",0
3,la causa no imputable debe ser entendida como ...,1
4,Al contratar un servicio de transporte de carg...,0
...,...,...
278,"""En virtud de lo expuesto, en el presente caso...",1
279,"""Sobre el particular, esta Sala no coincide co...",0
280,"""De esta manera, al no haber quedado acreditad...",1
281,"""Dicho esto, este Colegiado considera que el f...",1


In [6]:
#visualizamos conteo

DF_criterios["Decisión"].value_counts() #balanceado

1    158
0    125
Name: Decisión, dtype: int64

## 2. Limpieza

In [7]:
# quitamos caracteres especiales y numeros. Además quitamos las palabras pequeñas
# y un par de stopwords que no deberían estar (por obvias razones xD)

def limpieza(c):
    
    out = []
    c = re.findall(r"[a-zA-Z]+",c)
    for w in c:
        #aqui creo la lista
        if (len(w)>3 and (w!="para") and (w!="ademas") and (w!="culo")):
            out.append(w.lower())
    
    #aqui uno la lista en un cadena separada por espacios
    return " ".join(out)

In [8]:
# aplicar funcion de limpieza y creamos columna Limpio

DF_criterios['Limpio']=DF_criterios['Criterios aplicables'].astype("str").apply(limpieza)
DF_todas['Limpio']=DF_todas['todo'].astype("str").apply(limpieza)

In [9]:
#Quitamos stopwords predeterminadas de NLTK

def stopw(c):
    out=[]
    stop = stopwords.words("spanish") 
    for w in c.split():
        if w not in stop:
            out.append(w)
    return " ".join(out)

In [10]:
#aplicamos funcion de stopwords y creamos nueva columa SinSW

DF_criterios['SinSW']=DF_criterios['Limpio'].astype("str").apply(stopw)
DF_todas['SinSW']=DF_todas['Limpio'].astype("str").apply(stopw)

In [11]:
#creamos funcion para stemmer
from nltk.stem import SnowballStemmer

def Stemmer(c):
    out=[]
    spanish_stemmer = SnowballStemmer('spanish')
     
    for w in c.split():
        word_stem = spanish_stemmer.stem(str(w))
        out.append(word_stem)
    return " ".join(out)

In [12]:
#aplicamos funcion de stemmer
DF_criterios['ConStem']=DF_criterios['SinSW'].astype("str").apply(Stemmer)
DF_todas['ConStem']=DF_todas['SinSW'].astype("str").apply(Stemmer)

## 3. Vectorizacion

### 3.1. TF-IDF

In [13]:
#pasar a lista
criterios = DF_criterios['ConStem'].values.tolist()
todas_variables = DF_todas['ConStem'].values.tolist()

In [14]:
#vectorizacion para criterios aplicables

vectorizer1 = TfidfVectorizer(max_features=100)
FV1 = vectorizer1.fit_transform(criterios)
FV1 = FV1.toarray()

In [15]:
np.savetxt("TF-IDF_Vectorization_Criterios.csv",FV1, delimiter=",")

In [16]:
#mostramos palabras mas usadas
corpus1 = vectorizer1.get_feature_names()
corpus1

['acredit',
 'adecu',
 'administr',
 'apel',
 'aplic',
 'bien',
 'bolet',
 'calid',
 'cas',
 'caus',
 'comisi',
 'condicion',
 'conduct',
 'confirm',
 'conform',
 'consecuent',
 'consider',
 'consumidor',
 'conten',
 'contrat',
 'correct',
 'correspond',
 'criteri',
 'cuent',
 'cumpl',
 'custodi',
 'deb',
 'decl',
 'decret',
 'defect',
 'denunci',
 'derech',
 'destin',
 'determin',
 'dich',
 'efect',
 'ello',
 'embarg',
 'empres',
 'encomiend',
 'entreg',
 'equipaj',
 'establec',
 'evit',
 'exim',
 'existent',
 'expuest',
 'extrav',
 'extravi',
 'extrem',
 'final',
 'hab',
 'hech',
 'idon',
 'impon',
 'indecopi',
 'infracci',
 'infractor',
 'legisl',
 'materi',
 'mayor',
 'med',
 'medi',
 'merc',
 'mism',
 'mont',
 'mult',
 'necesari',
 'norm',
 'objet',
 'obligaci',
 'ofrec',
 'orden',
 'part',
 'pasajer',
 'pes',
 'present',
 'prest',
 'prestaci',
 'proced',
 'product',
 'protecci',
 'proveedor',
 'pued',
 'rdid',
 'realiz',
 'resoluci',
 'respect',
 'respons',
 'result',
 'sal',
 's

In [17]:
#vectorizacion para todas las variables

vectorizer2 = TfidfVectorizer(max_features=100)
FV2 = vectorizer2.fit_transform(todas_variables)
FV2 = FV2.toarray()

In [18]:
#mostramos el corpus
corpus2 = vectorizer2.get_feature_names()
corpus2

['acredit',
 'adecu',
 'administr',
 'aplic',
 'bien',
 'bolet',
 'calid',
 'carg',
 'cas',
 'caus',
 'civil',
 'comisi',
 'condicion',
 'conduct',
 'confirm',
 'conform',
 'consecuent',
 'consider',
 'consumidor',
 'conten',
 'contrat',
 'correct',
 'correspond',
 'cuent',
 'cumpl',
 'custodi',
 'deb',
 'decret',
 'defect',
 'denunci',
 'destin',
 'determin',
 'dich',
 'dig',
 'efect',
 'ello',
 'empres',
 'encomiend',
 'entreg',
 'equipaj',
 'establec',
 'exim',
 'existent',
 'expres',
 'extrav',
 'extrem',
 'facultad',
 'final',
 'francisc',
 'general',
 'hab',
 'hech',
 'idon',
 'impon',
 'indecopi',
 'infracci',
 'infractor',
 'juan',
 'legisl',
 'luis',
 'mayor',
 'med',
 'medi',
 'merc',
 'mism',
 'mult',
 'nacional',
 'norm',
 'objet',
 'ofrec',
 'orden',
 'organizaci',
 'part',
 'pasajer',
 'pes',
 'present',
 'prest',
 'prestaci',
 'proced',
 'product',
 'protecci',
 'proveedor',
 'pued',
 'rdid',
 'reglament',
 'resoluci',
 'respect',
 'respons',
 'result',
 'sal',
 'sancion

In [19]:
np.savetxt("TF-IDF_Vectorization_Todas.csv",FV2, delimiter=",")

## 3.2. Embeddings

In [20]:
from gensim.models.keyedvectors import KeyedVectors

"""
https://github.com/dccuchile/spanish-word-embeddings
FastText embeddings from SUC
Vector format (.vec) (3.4 GB)
"""

wordvectors_file_vec = 'embeddings-l-model.vec'
cantidad = 100000

wordvectors = KeyedVectors.load_word2vec_format(wordvectors_file_vec, 
                                                limit=cantidad)

In [21]:
def repr_emb(sentence):
    sentence = stopw(sentence) # eliminar stopwords
    sentence = limpieza(sentence) # eliminar la puntuación
    tokenized = re.split("\s+", sentence) # tokenizar
    # hallar la representación vectorial para cada palabra que pertenezca
    # al vocabulario
    list_emd = [wordvectors[x] for x in tokenized if x in wordvectors.vocab]
    if list_emd:
        # promediar los embeddings de las palabras
        numb_elems = len(list_emd)
        list_emd = np.array(list_emd).reshape(300, numb_elems).T.mean(axis=0)
        return list_emd
    # si ninguna palabra pertenece al vocabulario, retornar un vector zero
    else: 
        return np.zeros((300))

In [22]:
sentence = DF["Criterios aplicables"][1]
repr_emb(sentence).shape

(300,)

In [23]:
DF_emb = pd.DataFrame()

In [24]:
# hallar word embeddings para los critarios aplicables de todas las resoluciones
DF_emb["crit_apl_emb"] = DF["Criterios aplicables"].apply(repr_emb)

In [31]:
Stakc_emb = np.stack(DF_emb["crit_apl_emb"].values)

In [32]:
np.savetxt("Criterios_emb.csv",Stakc_emb, delimiter=",")